## Controlling Data Step Processing
### Understanding Data Step<font color='red'>(putlog)</font>
**Complilation**
- Check for syntax errors;
- Create the program data vector (PDV);
- Establish rules for processing data in the PDV;
- Create description portion of output table.
- **<font color='red'>Keep,Where, and Format</font>** are compile-time statements. They set rules for execution.

**Execution**
- Read Data;
- Process data in PDV;
- Output data to new table;

**Putlog Syntax**
```SAS
    putlog "NOTE: PDV after SET statement";
	putlog _all_; 
    putlog Ocean=;        /* Write the length of Ocean column to log*/
```
**Example**(p201p02)
```SAS
data np_parks;
	set pg2.np_final;
	putlog "NOTE: Start data step Iteration";      /* Write a color_coded note to the log */
	keep Region ParkName AvgMonthlyVisitors Acres Size;
	length Size $ 6;
    where Type="PARK";
	format AvgMonthlyVisitors Acres comma10.;
    Type=propcase(Type);
    putlog Type=;                                 /* Write the length of Type to the log */
	AvgMonthlyVisitors=sum(DayVisits,Campers,OtherLodging)/12;
	if Acres<1000 then Size="Small";
	else if Acres<100000 then Size="Medium";
	else Size="Large";
	putlog _all_;                               /* Write the contents of the PDV to the log */
run;
```

### Directing Data Step Output <font color='red'>(drop & output)</font>
```SAS
data table1(<drop/keep=col_name>) <table2,...>;   /* drop/keep columns from table */
    set input_data (<drop/keep=col_name>);        /* drop/keep columns from all table */
    .....
    output table1;                           /* output to table1 */
    .....
run;
```
**Example** (p201p03),(p201d02)
```SAS
data monument(drop=ParkType) park(drop=ParkType) other;
	set pg2.np_yearlytraffic(drop=region);
	if substr(ParkType,10,8) = "Park    " then output park;
	else if substr(ParkType,10,8) ="Monument" then output monument;
	else output other;
run;
```

## Summarizing Data
### Creating an Accumulating Column <font color='red'>(retain)</font>
- Using retain 
```SAS
...
retain col_name <initial_value>;  /* initialize value and retain the value */
...
```
Missing value plus value equals to missing value. <font color='red'>**sum**</font> function can solve this problem. 
- Using column
```SAS
accum_colum + colum_add          /* column_add will be accumating added and assign to accum_colum */
```
In this way, the missing value will be ignored.
**Example**(p202p01)
```SAS
data parkTypeTraffic;
	set pg2.np_yearlyTraffic;
	where ParkType in("National Monument" "National Park");
	if ParkType="National Monument" then MonumentTraffic + Count;
	if ParkType="National Park" then ParkTraffic + Count;
	format MonumentTraffic ParkTraffic comma10.;
run;
```

### Processing Data in Groups<font color='red'>(first/last.col_name)</font>
```SAS
data out_file;
    set in_file;
    by col_name1 col_name2; /* sort by col_name1 first, then sort by col_name2 within col_name group */
    first.col_name1 ...    /* temporary column generated by group col_name1 */
    last.col_name1 ...     /* temporary column generated by group col_name2 */
```
**Example**
```SAS
data storm2017_max;
	set storm2017_sort;
	by Basin;
	if last.basin=1;     /* put this statement just before run statement will obtain same results */ 
	StormLength=EndDate-StartDate;
	MaxWindKM=MaxWindMPH*1.60934;
run;
```

### Creating an Accumulating Column within Groups
- p202d03
```SAS
data houston_monthly;
	set pg2.weather_houston;
	keep Date Month DailyRain MTDRain;
	by Month; 
	if first.Month=1 then MTDRain=0;    /* reset the initial value for new group */
	MTDRain+DailyRain;
run;
```
Example
```SAS
data houston_monthly;
	set pg2.weather_houston;
	keep Date Month DailyRain MTDRain;
	by Month;
	if first.Month=1 then MTDRain=0;
	MTDRain+DailyRain;
	if last.Month=1 then output;     /* Equivalent:   if last.Month=1;  /   if last.Month;   */
run;       
```
- p202p04
```SAS
data TypeTraffic(keep=ParkType TypeCount);
    set work.sortedtraffic;
    by ParkType;
    if first.ParkType=1 then TypeCount=0;  /* initialize value for new group */
    TypeCount + Count;                     /* create accumating column */
	if last.ParkType;                      /* Output the last row of each group */
	format TypeCount comma12.;
run;
```
```SAS
data profitsummary(keep=region product sales returns profit totalProfit);
	set work.sortedshoes;
	by region product;
	profit=sales - returns; 
	if first.product=1 then TotalProfit=0;
	TotalProfit + Profit;
	if last.product=1;
	format TotalProfit dollar12.;
run;
```
<font color='red'>**Note**: Calculated columns or values cannot be used in where statement to filter rows.

## Manipulating Data with Functions
### <font color='red'>Columns lists</font>
```SAS
function(col_name1, col_name2,...); /* list the name of columns */
function(col1--coln);               /* Physical position from col1 to col2 */
function(of col_list);              /* col1_coln, where n is a sequential number; */
function(of A:);                    /* all columns start with A */

FORMAT col_list format.;
format _numeric_ 3.1;               /* _character_     _all_   */
```
- p203a01
```SAS
data quiz_summary;
	set pg2.class_quiz;
	Name = upcase(Name);
	Mean1=mean(Quiz1, Quiz2, Quiz3, Quiz4, Quiz5);
	Mean2=mean(of Quiz1-Quiz5);
	Mean3=mean(of Q:);        /* Mean1, Mean2, Mean3 generat same values */
run;
```

### <font color='red'>Call Routines</font>
```SAS
Call routine (argument_1 <, argument_2,....>);
```
**Example**
```SAS
data quiz_report;
	set pg2.class_quiz;
    call sortn(of Quiz1-Quiz5);    /* call sortn */ 
    QuizAvg = mean(of Quiz1-Quiz5);
run;
```
- p203a02
```SAS
data quiz_report;
    set pg2.class_quiz;   /* call missing(var1,var2...): assign vars columns as missing value */
	if Name in("Barbara", "James") then call missing(of Q:); 
run;
```

### Numeric Functions (<font color='red'>rand, largest, round, ceil, floor, int</font>)
```SAS/font
rand('distribution',parameter1,...);   /* assign a random number to a column */
largest(k,value_1 <,value_2....>);     /* Identify the top of column */
round(number <,rounding_unit>);        /* rounding_unit: .1,  .01,   */
ceil(number);                          /* smallest integer that is >= number 上进位 */
floor(number);                         /* greatest integer that is =< number 下进位*/
int(number);                           /* the integer number */
```
- p203d01 (p203a03)
```SAS
data quiz_analysis;
	StudentID = rand("interger",1000,9999);
	set pg2.class_quiz;
	drop Quiz1-Quiz5 name;
    Quiz1st = largest(1, of Quiz1-Quiz5);
    Quiz2nd = largest(2, of Quiz1-Quiz5);
    Quiz3rd = largest(3, of Quiz1-Quiz5);
    Top3Avg = round(mean(Quiz1st,Quiz2nd,Quiz3rd), .1);
run;
```

###  Data Functions (<font color='red'> intck, intnx, datepart, timepart, largest</font>)
```SAS
/* Return interval between two datetime */
/* interval: year/month/week/weekday/hour   method: D(discrete, default), C(countious) */
intck('interval',start_date,end_date <, 'method'>); 

/* shift the date */
/* increment: number of intervals to shift;   alignment:first(default)/middle/end/same */
intnx('interval',start,increment <, 'alignment'>); 

datepart(datetime_value);
timepart(datetime_value);

largest(k,col_list);    /* return the k-th largest among in col_liost */
```
- p203a04
```SAS
data storm_length;
	set pg2.storm_final(obs=10);
	keep Season Name StartDate Enddate StormLength Weeks;
	Weeks=intck('week', StartDate, EndDate, "C"); /* How many weeks between these two date */
run;
```
- p203d02
```SAS
data storm_damage2;
	set pg2.storm_damage;
	keep Event Date AssessmentDate Anniversary;
	AssessmentDate = intnx('month', Date, 2,'middle'); /* shift Date 2 month to the middle of new month */
	Anniversary = intnx('year',Date,10,'same');  /* shift Date 10 year at the same date */
    format Date AssessmentDate Anniversary date9.;
run;
```
- p203p02
```SAS
data rainsummary;
	set pg2.np_hourlyrain;
	by Month;
	if first.Month=1 then MonthlyRainTotal=0;
	MonthlyRainTotal+Rain;
	if last.Month=1;
	Date = datepart(DateTime);
	MonthEnd = intnx('month',Date,0,'end'); /* shift to the end of the month */
	format Date MonthEnd date9.;
	drop Rain DateTime;
run;
```
- p203p01
```SAS
data stays(keep=Park Stay: Avg);
	set pg2.np_lodging;
	Stay1 = largest(1, of CL:);
	Stay2 = largest(2, of CL:);
	Stay3 = largest(3, of CL:);
	Avg = round(mean(of CL:));
	if Avg > 0;               /* only output rows of avg>0 */
	format Stay: comma11.;
run;
```


### Character Function
#### <font color='red'>compbl, compress, strip, scan, find</font>
```SAS
compbl(col_name);   /* convert multiple blanks to single blank */
compress(col_name <,'characters'>); /* remove specified characters */
strip(col_name);   /* remove leading and trailing blanks */
scan(col_name,n <,'delimiters'>);   /* n: word to extract; '':characters that separate words */
find(col_name,substring <,'modifiers'>); /*return start position number, modifier: I(insensitive) T(trim) */
```
- p203a06
```SAS
data weather_japan_clean;
    set pg2.weather_japan;
    NewLocation= compbl(Location);
    NewStation= compress(Station,'-');
run;
```
- p203d03
```SAS
data weather_japan_clean;
	set pg2.weather_japan;
	Location = compbl(Location);
	City = propcase(scan(Location, 1,',')," "); /* space will treat as the seperate for propcase word */
	Prefecture = scan(Location, 2,',');
	Country = scan(Location,-1,',');            /* the last word in the string */
run;
```
- p203a07
```SAS
data weather_japan_clean;
	set pg2.weather_japan;
	Location=compbl(Location);
	City=propcase(scan(Location, 1, ','), ' ');
	Prefecture=strip(scan(Location, 2, ','));
	putlog Prefecture $quote20.;
	if Prefecture="Tokyo";
run;
```
- p203a08
```SAS
data storm_damage2;
	set pg2.storm_damage;
	CategoryLoc=find(upcase(Summary), 'CATEGORY');
	if CategoryLoc > 0 then Category=substr(Summary,CategoryLoc, 10);
run;
```
- p203p04

#### <font color='red'>length, anydigit, anyaloha, anypunct</font>
```SAS
length(col_name);   /* return the length of a non-black character string */
anydigit(col_name); /* return the first position at which a digit is found in the string */
anyalpha(col_name); /* return the first position at which an alpha character is found in the string */
anypunct(col_name); /* return the first position at which punctuation character is found in the string */
```

#### <font color='red'> tranwrd</font>
```SAS
tranwrd(col_name,'string','rep_string'); /* repace the string with rep_string in col_name column */
```

#### <font color='red'> cat, cats, catx</font>
```SAS
cat(string1,...string); /* concatenates strings together, does not remove leading or trailing blanks */
cats(string1,...stringn); /* concatenates strings together,  remove leading or trailing blanks  */
/* concatenates strings together,remove leading or trailing blanks,insert the delimiter between each string */
catx('delimiter',string1,...stringn); 
```
- p203a09
```SAS
data storm_id;
	set pg2.storm_final;
	keep StormID: ;
	Day=StartDate-intnx('year', StartDate, 0);
	StormID1=cat(Name, Season, Day);
	StormID2=cats(Name, Season, Day);
	StormID3=catx('-',Name, Season, Day);
run;
```
- p203p05
```SAS
data parks;
	set pg2.np_monthlytraffic;
	where ParkName like '%NP';
	Park = substr(ParkName,1,find(ParkName, 'NP')-2);
	Location = compbl(propcase(Location));
	Gate = tranwrd(Location,'Traffic Count At','');
	GateCode = catx('-',ParkCode,Gate);
run;
```

### Convert Column Type (<font color='red'>input,put,rename</font>)
```SAS
input(col_name, informat);  /* character to numeric; informat: how the data in the input table is displayed */
put(col_name,format);   /* numberic to character */
```
|Character Value to Convert | Informat for the Input Function | Numberic Value Returned |
|---------|-------|-------|
|15OCT2018  | Date9.    |21472|
|10/15/2018 | MMDDYY10. |21472|
|15/10/2018 | DDMMYY10. |21472|
|123,456.78 | Comma12.  |123456.78|
|$123,456.78| Dollar.   |123456.78|
|123456     |6.         |123456|


|Numeric Value to Convert | Informat for the Put Function | Character Value Returned |
|---------  |-------|-------|
|21472      | Date9.      |15OCT2018|
|21472      | Downame3.   |MON|
|21472      | Year4.      |2018|
|123456.78  | Comma10.2   |123,456.78|
|123456.78  | Dollar.11.2 |$123,456.78|
|123.456    | 6.          |123.46|

- Informat <font color='red'>**anydtdtew.**</font> can be used to read a variety dates. Be careful if the date is ambiguous. 
```SAS
options datestyle=MDY;   /* 06/01/2018 -> June 01,2018 */
options datestyle=DMY;   /* 06/01/2018 -> January 06,2018 */
```
- Change the column type of the input table need to rename the input column you want to change:
```SAS
table(rename=(current_col_name=new_col_name));
```
- p203a11
```SAS
data work.stocks2;
    set pg2.stocks2(rename=(Volume=CharVolume)); /* assign Volume to CharVolume */
    Date2 = input(Date,date9.);                  /* read Date with date9. informat and assign to Date2 */
 	Volume = input(CharVolume,comma12.);         
 	drop CharVolume;
run;
```

- p203d04

```SAS
data atl_precip;
	set pg2.weather_atlanta(rename=(date=CharDate));
	where AirportCode = 'ATL';
	drop AirportCode City Temp: ZipCode Precip CharDate;
	if Precip ne 'T' then PrecipNum = input(Precip,6.);
	else PrecipNum = 0;
	TotalPrecip + PrecipNum;
	Date = input(CharDate,mmddyy10.);
	format Date date9.;
run;

data atl_precip;
	set pg2.weather_atlanta;
	CityStateZip = catx(' ',City,'GA',ZipCode);
	ZipCodeLast2 = substr(put(ZipCode,z5.),4,2);  
    /* z5. will write a number as a character string and insert leading zeros to fill 5 total positions */
run;
```
- p203a13
```SAS
data stocks2;
   set pg2.stocks2(rename=(Volume=CharVolume Date=CharDate));
   Volume = input(CharVolume,comma12.);
   Date = input(CharDate,date9.);
   drop CharVolume CharDate;
run;
```
**Note**: Convert columns to same type before manipulate will be better.

## Creating and Using Custom Formats
### Formatting Data Values (<font color='red'> proc format)
```SAS
proc format;
    value format_name value_or_range_1 = 'formatted_value'
                      value_or_range_2 = 'formatted_value'
                      ...;
run;
```
- format_name: 
    - Character formats: $ followed by letter or underscore
    - Numeric formats: begin with letter or underscore
    - upto 32 characters long
    - cannot end in number or match existing SAS formate
    
**Example** 
```SAS
/* create format */
proc format;
    value $genfmt 'F'='Female'
                  'M'='Male';
run;

/* apply format */
proc print data=pg2.class_birthdate;
    format Gender $genfmt.;
run;
```
- p204a02

```SAS
proc format;
    value $genfmt 'F'='Female'
                  'M'='Male'
                  other = 'Miscoded';        /* all values that don't mathch any other value */
    value hrange low-< 58 = 'Below Average' /* lowest possible value */
                 58 - 60 = 'Average'
                 60 <- high = 'Above Average' ; /* highest possible value */
run;
```
- p204d01

```SAS
proc format;
    value stdate low - '31DEC1999'd = '1999 and before'
                 '01JAN2000'd - '31DEC2009'd = '2000 to 2009'
                 '01JAN2010'd - high = '2010 and later'
                 . = 'Not Supplied';
	value $region 'NA'='Atlantic'
				  'WP','EP','SP'='Pacific'
				  'NI','SI'='Indian'
				  ' '='Missing'
				  other='Unknow';
run;
```
- p204a03 (p204p01,p204p02)

```SAS
data storm_summary;
    set pg2.storm_summary;
    Basin=upcase(Basin);
    BasinGroup = put(Basin,$region.);
run;
```

### Creating Custom Formats from Tables (<font color='red'> cntlin,retain </font>)

```SAS
data work.sbdata;
    retain FmtName '$sbfmt';   /* create the FmtName column and retain the value sbfmt for each row */
    set pg2.storm_subbasincodes;
run;

proc format cntlin=work.sbdata; /* create format from catdata table */
run;

data catdata;
    retain FmtName 'catfmt';
    set pg2.storm_categories (rename=(Low=Start High=End Category=Label));
    keep FmtName Start End Label;
run;

proc format cntlin=catdata;   /* create format from catdata table */ 
run;
```


- **Create perminate customer formats**
```SAS
proc format library=pg2.myfmts;
```

- p204a04
```SAS
proc format fmtlib library=work;   /* save formats to work.formats */
	select $sbfmt catfmt;          /* only includes sbfmt and catfmt */
run;
```

### Save and use customer formats folder(<font color='red'>fmtsearch</font>)
```SAS
options fmtsearch=(pg2.myfmts sashelp); /* search in myfmts of pg2 library and formats catalog of sashelp library */
```
- p204a05

```SAS
proc format library=pg2;   /* save the formats to the pg2.formats */
    value $gender 'F'='Female'
                  'M'='Male'
                  other='Miscoded';
    value hght low-<58  = 'Below Average'
                58-60   = 'Average'
               60<-high = 'Above Average';
run;

options fmtsearch=(pg2);   /* direct SAS where to find the permanent formats */
proc print data=pg2.class_birthdate noobs;
    where Age=12;
    var Name Gender Height;
    format Gender $gender. Height hght.;
run;
```
- p204p04

```SAS
data type_lookup;
	retain FmtName '$TypeFmt';
    set pg2.np_codeLookup (rename=(ParkCode=Start Type=Label));
    keep Start Label FmtName;
run;

proc format cntlin=type_lookup;
run;

title 'Traffic Statistics';
proc means data=pg2.np_monthlyTraffic maxdec=0 mean sum nonobs;
    var Count;
    class ParkCode Month;
    label ParkCode='Name';
    format ParkCode $TypeFmt.;
run;
title;
```
**Summary** p204p05

```SAS
data np_lookup;
	retain FmtName '$RegLbl';
    set pg2.np_codeLookup (rename=(ParkCode=Start));
    length label $20.;
	if region =' ' then label='Unknown';
	else label = region;
	keep start label FmtName;
run;

proc format cntlin=np_lookup;
run;

data np_endanger;
    set pg2.np_species;
    where Conservation_Status='Endangered';
    region = put(ParkCode, $RegLbl.);
run;

title 'Number of Endangered Species by Region';
proc freq data=np_endanger;
    tables Region / nocum;
run;
title;
```

## Combing Tables
### Concatenating Tables
```SAS
DATA output_table;
SET input_table1(rename=(current_colname=new_colname))
    input-table2 ...; 
RUN;
```
- Columns that are not in all tables are also included in the output table.
- **Rename** used to rename columns in one or both tables so that they align in the new table.
- p205d01
```SAS
data storm_complete;
	set pg2.storm_summary 
		pg2.storm_2017(rename=(Year=Season) drop=Location); 
	Basin=upcase(Basin);
run;
```
- p205a01
```SAS
data class_current;
	length Name $9;      /* keep the length of column in two tables are same */
	set sashelp.class 
  	  	pg2.class_new2(rename=(Student=Name));
run;
```
- p205p01
```SAS
data work.np_combine;
    set pg2.np_2014(rename=(Park=ParkCode Type=ParkType))
    	pg2.np_2015 
    	pg2.np_2016;
    CampTotal = sum(of Camping:);
    where Month in(6,7,8) and ParkType="National Park";
    format CampTotal comma15.;
    drop Camping:;
run;
```

### Merging Tables (<font color='red'> merge...by </font>)

```SAS
PROC SORT DATA=input_table 
    OUT=output_table; 
RUN;

DATA output_table;
    MERGE input_table1 input_table2 ...;
    BY by_column(s);
RUN;
```
- Any tables listed in the Merge statement must be sorted by the same column listed in the BY statement.
- The **Merge** statement combines rows where the by_column values match.
- p205a02

```SAS
proc sort data=pg2.class_teachers out=teachers_sort;
	by Name;
run;

proc sort data=pg2.class_test2 out=test2_sort;
	by Name;
run;

data class2;
	merge teachers_sort test2_sort;
	by Name;
run;
```
- p205d02

```SAS
proc sort data=pg2.storm_summary out=storm_sort;
	by Basin;
run;

proc sort data=pg2.storm_basincodes out=basincodes_sort;
	by BasinCode;
run;

data storm_summary2;
	merge storm_sort basincodes_sort(rename=(BasinCode=Basin));
	by Basin;
run;
```

### Identifying Matching and Nonmatching Rows (<font color='red'> IN=var </font>)
```SAS
DATA output_table;
    MERGE input_table1(IN=var1) input_table2(IN=var2) ...;
    BY by_column(s);
RUN;
```
- **IN=var** is assigned a value of 0(this row was not included) or 1(this row was included).
- Use **IF** to subset rows based on matching or nonmatching rows.
- p205d03
```SAS
data damage_detail;
	merge storm_final_sort  storm_damage(in=inDamage);
	by Season Name;
	if inDamage=1;
	keep Season Name BasinName MaxWindMPH MinPressure Cost Deaths;
run;
```
- p205a04/p205p03
```SAS
data damage_detail storm_other(drop=Cost Deaths);
	merge storm_final_sort(in=inFinal) storm_damage(in=inDamage);
	keep Season Name BasinName MaxWindMPH MinPressure Cost Deaths;
	by Season Name;
	if inDamage=1 and inFinal=1 then output damage_detail;
	else output storm_other;
run;
```

### Data step Merge and Proc SQl Join

|Data Step Merge | Proc SQL Join|
|---------------|--------------|
|requires sorted input data | does not require sorted data|
|efficent,sequential processing | matching columns do not need the same name|
|can create multiple output tables for matches and nonmatches in one step | easy to define complex matching criteria between multiple tables in a single query|
|provides additional complex data processing syntax | can be used to create a Cartesian product for many to many joins|

## Processing Repetitive Code
### <font color='red'>Do To</font>
```SAS
DO index_column = start TO stop <BY increment>; 
    . . . repetitive code . . .
    <output;>
END;
```
- p206d02/p206a01/p206p02
```SAS
data YearSavings;
	set pg2.savings;
	Savings = 0;
	do Year=1 to 5;
		do Month=1 to 12;
			Savings+Amount;
			Savings+(Savings*0.02/12);
			output;
		end;	
	end;
	format Savings comma12.2;
run;
```

### <font color='red'>Do Until | While</font>
```SAS
DO UNTIL | WHILE (expression);
    . . . repetitive code . . .
    <OUTPUT;> 
END;
```
- p206a02

```SAS
data Savings3K;
   set pg2.savings;
   Month=0;
   Savings=0;
   do until (Savings>3000);
      Month+1;
      Savings+Amount;
      Savings+(Savings*0.02/12);
   end;
   format Savings comma12.2;
run;
/*********** Way 2 **************/
data Savings3K;
   set pg2.savings;
   Month=0;
   Savings=0;
   do while (Savings=<3000);
      Month+1;
      Savings+Amount;
      Savings+(Savings*0.02/12);
   end;
   format Savings comma12.2;
run;
```

### <font color='red'>Do To Until | While</font>
```SAS
DO index_column = start TO stop <BY increment> UNTIL | WHILE (expression);
    . . . repetitive code . . . 
END;
```
- p206p04
```SAS
data IncreaseDayVisits;  
    set pg2.np_summary;
    where Reg='NE' and DayVisits<100000;
    IncrDayVisits=DayVisits;
    Year = 0;
	do until (IncrDayVisits >= 100000);
    IncrDayVisits=IncrDayVisits*1.06;
    Year + 1;
	end;
	output;
    format IncrDayVisits comma12.;
    keep ParkName DayVisits IncrDayVisits Year;
run;
```
- p206p05
```SAS
data IncrExports;
    set pg2.eu_sports;
    where Year=2015 and Country='Belgium' 
          and Sport_Product in ('GOLF','RACKET');
	do Year=2016 To 2025 while(Amt_Export <= Amt_Import);
    Amt_Export=Amt_Export*1.07;
    output;
	end;
    format Amt_Import Amt_Export comma12.;
run;
```

**Summary**
- <font color='red'>**Do**</font>: The index will be an increment **beyond** the stop value;
- <font color='red'>**Until**</font>: The condition is checked at the **bottom** of the loop; A DO UNTIL loop always executes **at least one time**.
- <font color='red'>**While**</font>: The condition is checked at the **top** of the loop; A DO WHILE loop **does not iterate even once** if the condition is initially false.

## Restructuring Tables
### With Data step
- Assignment statements are used to create new columns for stacked values;
- Explict OUTPUT statement is used to create multiple rows for each input row;
- Do loops can be nested;
#### Wide Table to Narrow Table

- p207p01 (Wide Table to Narrow Table)
```SAS
data work.camping_narrow(drop=Tent RV Backcountry);
	set pg2.np_2017Camping;
	length CampType $12;
	format CampCount comma12.;
	CampType='Tent';
	CampCount=Tent;
	output;
	CampType='RV';
	CampCount=RV;
	output;
	CampType='Backcountry';
	CampCount=Backcountry;
	output;
run;
```
#### Narrow Table to Wide Table
- p207a02 (Narrow Table to Wide Table)

```SAS
data work.camping_wide;
    set pg2.np_2016Camping;
    by ParkName;
    keep ParkName Tent RV Backcountry;
    format Tent RV Backcountry comma12.;
    retain ParkName Tent RV Backcountry;
    if CampType='Tent' then Tent=CampCount;
    else if CampType='RV' then RV=CampCount;
    else if CampType='Backcountry' then Backcountry=CampCount;
    if last.ParkName;
run;

data class_wide;
    set pg2.class_test_narrow;
    by Name;
    retain Name Math Reading;
    keep Name Math Reading;
    if TestSubject="Reading" then Reading=TestScore;
    else if TestSubject="Math" then Math=TestScore;
    if last.Name=1 then output;
run;
```

### With <font color='red'>**proc transpose**</font>
```SAS
PROC TRANSPOSE DATA=input_table OUT=output_table 
    PREFIX=column <NAME=column>;
    <VAR columns(s);> <ID column;>
    <BY column(s);>
RUN;
```
- Output table will include a separate column for each value of the ID column;
- **Var** statement lists the columns to be transposed;
- **BY** statement transposes data within groups;
- **Prefix** provides a prefix for each value of the ID columns in the output table;
- **NAME** names the column that identifies the source column containing the transposed values.
- by default, PROC TRANSPOSE transposes all the numeric columns

- p207d02(narrow to wide)
```SAS
/* 根据 ID 将 Var中的值拉平 */
proc transpose data=pg2.storm_top4_narrow 
    out=wind_rotate(drop=_name_) 
    /* _name_ is automaticely generate column, containing the column name of transpose value */
    prefix=Wind; /* Column name of Transposed Column Value 窄变宽后，列名起始字符 */
	var WindMPH; /* Transpose value 将该列值拉平*/
	id WindRank; /*Column used to indicate the number of transpose 根据该列反复拉平*/
	by Season Basin Name; /* 这些值保持不变*/
run;
```
- p207a05(wide to narrow)
```SAS
proc transpose data=pg2.storm_top4_wide 
	out=storm_top4_narrow
	name=WindRank   /* Column name of Transposed Column Names */
	prefix=WindMPH; /* Column name of Transposed Column Value */
	by Season Basin Name; /* Transposes data within groups */
	var Wind1 Wind2;   /* Transposed columns. All numerica columns are by default. */
run;
```
- p207p04(wide to narrow)
```SAS
/* 将 var列值合并拉直 */
proc transpose data=pg2.np_2017camping 
	out=work.camping2017_t(rename=(col1=Count)) /* 拉直后的列名为 count列 */
	name=Location;      /*  var 中的列名合并拉直为 Location 列 */
	by ParkName;
	var Tent RV;
run;
```